In [36]:
# Init Memcached Snippet

import socket

def init_memcached(server:str, port:int):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((server, port))
    data_block = 4096 * 'x'
    key_max = 10000
    for i in range(key_max+1):
        s.sendall(f'set memtier-{i} 0 0 4096 \r\n{data_block}\r\n'.encode())
        data = s.recv(64)
        assert(data=='STORED\r\n'.encode())
    s.close()
    
    print(f'initialized all keys on memcached server {server}:{port}')

init_memcached(server='localhost', port=11211)

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [14]:
for i in range(0):
    print(i)
    
print('here')

here


In [38]:
def get_c():
    with open('./scripts/experiments/exp21.json') as json_data_file:
        config = json.load(json_data_file)

    for key, value in config.items():
        if not isinstance(value, list):
            config[key] = [value]

    config['tmp'] = list(zip(config['n_middleware'], config['n_instances_mt_per_machine'], config['n_threads_per_mt_instance']))
    del config['n_middleware']
    del config['n_instances_mt_per_machine']
    del config['n_threads_per_mt_instance']

    for c in (dict(zip(config, x)) for x in itertools.product(*config.values())):
        c['n_middleware'] = c['tmp'][0]
        c['n_instances_mt_per_machine'] = c['tmp'][1]
        c['n_threads_per_mt_instance'] = c['tmp'][2]
        del c['tmp']
        yield(c)

i = 0
for config in get_c():
    i += 1
    print(i)
    print(config)

1
{'n_server': 1, 'n_client': 3, 'n_vc': 1, 'workload_ratio': '1:0', 'multi_get_behaviour': None, 'multi_get_size': None, 'n_worker_per_mw': 0, 'repetitions': 3, 'n_middleware': 0, 'n_instances_mt_per_machine': 1, 'n_threads_per_mt_instance': 2}
2
{'n_server': 1, 'n_client': 3, 'n_vc': 1, 'workload_ratio': '0:1', 'multi_get_behaviour': None, 'multi_get_size': None, 'n_worker_per_mw': 0, 'repetitions': 3, 'n_middleware': 0, 'n_instances_mt_per_machine': 1, 'n_threads_per_mt_instance': 2}
3
{'n_server': 1, 'n_client': 3, 'n_vc': 4, 'workload_ratio': '1:0', 'multi_get_behaviour': None, 'multi_get_size': None, 'n_worker_per_mw': 0, 'repetitions': 3, 'n_middleware': 0, 'n_instances_mt_per_machine': 1, 'n_threads_per_mt_instance': 2}
4
{'n_server': 1, 'n_client': 3, 'n_vc': 4, 'workload_ratio': '0:1', 'multi_get_behaviour': None, 'multi_get_size': None, 'n_worker_per_mw': 0, 'repetitions': 3, 'n_middleware': 0, 'n_instances_mt_per_machine': 1, 'n_threads_per_mt_instance': 2}
5
{'n_server': 1

In [ ]:
# Connect to VM via SSH

import paramiko


host = 'storefvjdk4dylhf5ksshpublicip4.westeurope.cloudapp.azure.com'
username = 'kunicola'
key_filename= 'C:/Users/nicok/.ssh/asl_new/asl-private-openssh.ppk'
cmd = 'ls; cd experiments; ls'

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy()) 
ssh.connect(host, username=username, key_filename=key_filename)


ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(cmd)
print(ssh_stdout.readlines())



ssh.close()

In [1]:
import paramiko

def get_ssh_client(host):
    # TODO [nku] move to config file?
    username = 'kunicola'
    key_filename= 'C:/Users/nicok/.ssh/asl_new/asl-private-openssh.ppk'

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy()) 
    ssh.connect(host, username=username, key_filename=key_filename)
    return ssh

In [2]:
# load vm configs

import json

with open('vm_config.json') as json_data_file:
    vm = json.load(json_data_file)
print(vm['client'][0]['host'])

def format_stdout(stdout):
    for line in iter(stdout.readline, ""):
        print(f"\t{line}", end='')

storefvjdk4dylhf5ksshpublicip1.westeurope.cloudapp.azure.com


In [3]:
import time

def start_server(experiment_name, server_id, config):
    print(f"Starting Server {server_id} in Experiment {experiment_name}...")
    ssh = get_ssh_client(host=config['host'])

    print("  retrieving code from master...")
    stdin, stdout, stderr = ssh.exec_command("git -C ~/asl-fall18-project pull origin master")
    format_stdout(stdout)
    
    print("  git commit id...")
    stdin, stdout, stderr = ssh.exec_command("git -C ~/asl-fall18-project rev-parse HEAD")
    format_stdout(stdout)
    
    # TODO run as window
    print("  starting memcached...")
    stdin, stdout, stderr = ssh.exec_command(f"screen -dmS {server_id} memcached -l {config['ip']} -p {config['port']}")
    format_stdout(stdout)
    format_stdout(stderr)
    
    print("  initializing memcached...")
    stdin, stdout, stderr = ssh.exec_command(f"python3.6 asl-fall18-project/python/scripts/init_memcached.py -ip {config['ip']} -port {config['port']}")
    format_stdout(stdout)
    
    print(f"  listing screen windows...")
    stdin, stdout, stderr = ssh.exec_command("screen -list")
    format_stdout(stdout)
    
    print("  closing ssh client...")
    ssh.close()
    
    print("Finished Starting Server")
    
def stop_server(experiment_name, server_id, config):
    print(f"Stopping Server {server_id} in Experiment {experiment_name}...")
    ssh = get_ssh_client(host=config['host'])
    
    print("  listing screen windows...")
    stdin, stdout, stderr = ssh.exec_command("screen -list")
    format_stdout(stdout)
    
    print(f"  stopping server {server_id}...")
    stdin, stdout, stderr = ssh.exec_command(f"screen -S {server_id} -X quit")
    format_stdout(stdout)
    
    print("  listing screen windows...")
    stdin, stdout, stderr = ssh.exec_command("screen -list")
    format_stdout(stdout)

    print("  closing ssh client...")
    ssh.close()
    
    print("Finished Stopping Server")

In [ ]:
experiment_name = 'exp21'
server_id = 'server_00'
config = {
    'host': vm["server"][0]['host'],
    'ip' : 'localhost',
    'port' : 11211,
}

start_server(experiment_name, server_id, config)

time.sleep(20)
stop_server(experiment_name, server_id, config)


# killall memcached on failure

In [4]:
def start_client(experiment_name, client_id, config):
    print(f"Starting Client {client_id} in Experiment {experiment_name}...")
    ssh = get_ssh_client(host=config['host'])
    
    print("  creating experiment directory...")
    stdin, stdout, stderr = ssh.exec_command(f"mkdir -p ~/experiments/{experiment_name}/{client_id}/")
    format_stdout(stdout)
    format_stdout(stderr)
    
    print(f"  starting client in a detached screen window with id: {client_id}...")

        
    multikey_arg = "" if config['multi_get_size'] is None else f"--multi-key-get={config['multi_get_size']} "
        
    stdin, stdout, stderr = ssh.exec_command(f"cd ~/experiments/{experiment_name}/{client_id};\
                                                            screen -dmS {client_id} \
                                                             memtier_benchmark \
                                                                --threads={config['n_threads_per_mt_instance']} \
                                                                --clients={config['n_vc']} \
                                                                --ratio={config['workload_ratio']} \
                                                                --server={config['server']} \
                                                                --port={config['port']} \
                                                                {multikey_arg}\
                                                                --json-out-file={client_id}.log \
                                                                --protocol=memcache_text \
                                                                --key-maximum=10000 \
                                                                --data-size=4096 \
                                                                --expiry-range=9999-10000")
    format_stdout(stdout)
    format_stdout(stderr)  
    
    print(f"  listing screen windows...")
    stdin, stdout, stderr = ssh.exec_command("screen -list")
    format_stdout(stdout)
    
    print("  closing ssh client...")
    ssh.close()
    
    print("Finished Starting Client")
    
def stop_client(experiment_name, client_id, config):
    print(f"Stopping Client {client_id} in Experiment {experiment_name}...")
    ssh = get_ssh_client(host=config['host'])
    
    print("  listing screen windows...")
    stdin, stdout, stderr = ssh.exec_command("screen -list")
    format_stdout(stdout)
    
    print(f"  stopping client {client_id}...")
    stdin, stdout, stderr = ssh.exec_command(f"screen -S {client_id} -X quit")
    format_stdout(stdout)
    
    print("  listing screen windows...")
    stdin, stdout, stderr = ssh.exec_command("screen -list")
    format_stdout(stdout)

    print("  closing ssh client...")
    ssh.close()
    
    print("Finished Stopping Client")

In [ ]:
experiment_name = 'exp21'
client_id = 'client_00'
config = {
    'host': vm["client"][0]['host'],
    'server' : 'localhost',
    'port' : 11211,
    'n_threads_per_mt_instance' : 2,
    'n_vc':1,
    'workload_ratio': '1:1', # Set:Get ratio [read_only, write_only]
    'multi_get_size': None # 1
}

start_client(experiment_name=experiment_name, client_id=client_id, config=config)
time.sleep(10)
stop_client(experiment_name=experiment_name, client_id=client_id, config=config)

In [5]:
import time



def start_middleware(experiment_name, mw_id, config):
    print(f"Starting Middleware {mw_id} in Experiment {experiment_name}...")
    ssh = get_ssh_client(host=config['host'])

    print("  cleaning with ant...")
    stdin, stdout, stderr = ssh.exec_command("ant clean -buildfile ~/asl-fall18-project/java")
    format_stdout(stdout)

    print("  retrieving code from master...")
    stdin, stdout, stderr = ssh.exec_command("git -C ~/asl-fall18-project pull origin master")
    format_stdout(stdout)
    
    print("  git commit id...")
    stdin, stdout, stderr = ssh.exec_command("git -C ~/asl-fall18-project rev-parse HEAD")
    format_stdout(stdout)

    print("  building with ant...")
    stdin, stdout, stderr = ssh.exec_command("ant -buildfile ~/asl-fall18-project/java")
    format_stdout(stdout)
    
    print("  creating experiment directory...")
    stdin, stdout, stderr = ssh.exec_command(f"mkdir -p ~/experiments/{experiment_name}/{mw_id}")
    format_stdout(stdout)

    print(f"  starting middleware in a detached screen window with id: {mw_id}...")
    stdin, stdout, stderr = ssh.exec_command(f"cd ~/experiments/{experiment_name}/{mw_id};\
                                                            screen -dmS {mw_id} \
                                                             java -jar ~/asl-fall18-project/java/dist/middleware-kunicola.jar \
                                                                -l {config['ip']} \
                                                                -p {config['port']} \
                                                                -t {config['n_worker_per_mw']} \
                                                                -s {config['is_sharded']} \
                                                                -m {' '.join(config['server'])}")
    format_stdout(stdout)

    print(f"  listing screen windows...")
    stdin, stdout, stderr = ssh.exec_command("screen -list")
    format_stdout(stdout)
    
    print("  closing ssh client...")
    ssh.close()
    
    print("Finished Starting Middleware")

def stop_middleware(experiment_name, mw_id, config):
    print(f"Stopping Middleware {mw_id} in Experiment {experiment_name}...")
    ssh = get_ssh_client(host=config['host'])
    
    print("  listing screen windows...")
    stdin, stdout, stderr = ssh.exec_command("screen -list")
    format_stdout(stdout)
    
    print(f"  stopping middleware {mw_id}...")
    stdin, stdout, stderr = ssh.exec_command(f"screen -S {mw_id} -X quit")
    format_stdout(stdout)
    
    print("  listing screen windows...")
    stdin, stdout, stderr = ssh.exec_command("screen -list")
    format_stdout(stdout)

    print("  closing ssh client...")
    ssh.close()
    
    print("Finished Stopping Middleware")

In [ ]:
experiment_name = 'exp21'
mw_id = 'mw_00'
config = {
    'host': vm["middleware"][0]['host'],
    'ip': vm["middleware"][0]['private_ip'],
    'port' : 6379,
    'n_worker_per_mw' : 10,
    'is_sharded' : 'false',
    'server' : ['localhost:11211',  'localhost:11212', 'localhost:11213']
}

start_middleware(experiment_name, mw_id, config)

time.sleep(10)

stop_middleware(experiment_name, mw_id, config)

In [6]:
experiment_name = 'exp21'

# SERVER
server_id = 'server_00'
server_config = {
    'host': vm["server"][0]['host'],
    'ip' : vm["server"][0]['private_ip'],
    'port' : 11211,
}

start_server(experiment_name=experiment_name, server_id=server_id, config=server_config)

# MIDDLEWARE
mw_id = 'mw_00'
mw_config = {
    'host': vm["middleware"][0]['host'],
    'ip': vm["middleware"][0]['private_ip'],
    'port' : 6379,
    'n_worker_per_mw' : 10,
    'is_sharded' : 'false',
    'server' : [f"{vm['server'][0]['private_ip']}:11211"]
}

start_middleware(experiment_name=experiment_name, mw_id=mw_id, config=mw_config)

# CLIENT
client_id = 'client_00'
client_config = {
    'host': vm["client"][0]['host'],
    'server' : vm["middleware"][0]['private_ip'],
    'port' : 6379,
    'n_threads_per_mt_instance' : 2,
    'n_vc':1,
    'workload_ratio': '1:1', # Set:Get ratio [read_only, write_only]
    'multi_get_size': None # 1
}

start_client(experiment_name=experiment_name, client_id=client_id, config=client_config)


Starting Server server_00 in Experiment exp21...
  retrieving code from master...
	Already up-to-date.
  git commit id...
	9fd6f51669f9e88066fb41de812292e9274b0f87
  starting memcached...
  initializing memcached...
	initialized all keys on memcached server 10.0.0.11:11211
  listing screen windows...
	There is a screen on:
		4180.server_00	(10/15/2018 02:54:46 PM)	(Detached)
	1 Socket in /var/run/screen/S-kunicola.
  closing ssh client...
Finished Starting Server
Starting Middleware mw_00 in Experiment exp21...
  cleaning with ant...
	Buildfile: /home/kunicola/asl-fall18-project/java/build.xml
	
	clean:
	   [delete] Deleting directory /home/kunicola/asl-fall18-project/java/build
	   [delete] Deleting directory /home/kunicola/asl-fall18-project/java/dist
	
	BUILD SUCCESSFUL
	Total time: 1 second
  retrieving code from master...
	Updating 7a72bec..9fd6f51
	Fast-forward
	 python/scripts/init_memcached.py | 19 +++++++++++++++++++
	 1 file changed, 19 insertions(+)
	 create mode 100644 pyth

In [7]:
stop_middleware(experiment_name=experiment_name, mw_id=mw_id, config=mw_config)
stop_server(experiment_name=experiment_name, server_id=server_id, config=server_config)

Stopping Middleware mw_00 in Experiment exp21...
  listing screen windows...
	There is a screen on:
		2636.mw_00	(10/15/2018 02:55:05 PM)	(Detached)
	1 Socket in /var/run/screen/S-kunicola.
  stopping middleware mw_00...
  listing screen windows...
	No Sockets found in /var/run/screen/S-kunicola.
	
  closing ssh client...
Finished Stopping Middleware
Stopping Server server_00 in Experiment exp21...
  listing screen windows...
	There is a screen on:
		4180.server_00	(10/15/2018 02:54:45 PM)	(Detached)
	1 Socket in /var/run/screen/S-kunicola.
  stopping server server_00...
  listing screen windows...
	No Sockets found in /var/run/screen/S-kunicola.
	
  closing ssh client...
Finished Stopping Server
